# Projeto de Data Science: Analisando o engajamento do Instagram

## Qual tipo de conteúdo mais engaja no instagram da minha empresa?
## Qual tag mais engaja nas postagens?

##### Informações dadas pela empresa:
- Na base de dados do instagram temos informações desde que o usuário começou a postar na marca até o dia 27 de março
- Podem ignorar a coluna visualizações, queremos entender apenas curtidas, comentários e interações
- Tags vazias é que realmente não possuem tag (Tratar como vazio)

#### Vamos importar e visualizar a base

In [32]:
#Importando as bibliotecas
import pandas as pd
import numpy as np

In [33]:
#Impotando a base 
base = pd.read_excel(r'C:\Users\cacam\Downloads\Analisando o engajamento no Instagram.xlsx')

In [34]:
#Visualizando a base
base.head()

,Tipo,Data,Curtidas,Comentários,Visualizações,Tags,Pessoas,Campanhas,Carrossel,Interacoes
0,Foto,2021-09-11,2858,16,NaN,Loja,N,N,NaN,2874
1,Foto,2021-09-11,2930,28,NaN,Loja/Produtos,N,N,NaN,2958
2,Foto,2021-09-11,2807,9,NaN,Loja,N,N,NaN,2816
3,Vídeo,2021-09-12,5115,49,"82,878.00",Produtos,N,N,NaN,5164
4,Foto,2021-09-13,4392,45,NaN,Produtos,S,N,NaN,4437


In [35]:
#Apagando a coluna 'Visualizações'
base = base.drop('Visualizações', axis = 1)

In [36]:
#Tamanho da base
base.shape

(52, 9)

In [37]:
#Verificando as informações da base
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Tipo         52 non-null     object        
 1   Data         52 non-null     datetime64[ns]
 2   Curtidas     52 non-null     int64         
 3   Comentários  52 non-null     int64         
 4   Tags         44 non-null     object        
 5   Pessoas      52 non-null     object        
 6   Campanhas    52 non-null     object        
 7   Carrossel    8 non-null      object        
 8   Interacoes   52 non-null     int64         
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 3.8+ KB


### Tratando valores nulos

##### Entendendo os valores da coluna Carrossel

In [38]:
base.Carrossel.value_counts()

S    8
Name: Carrossel, dtype: int64

Como temos S para SIM, os valores nulos são das postagens que NÃO são carrossel. Sendo assim, vamos trocar o NaN para N (NÃO)

In [39]:
#Filtrando os valores em que carrossel é nulo
base.loc[base.Carrossel.isnull()].head()

,Tipo,Data,Curtidas,Comentários,Tags,Pessoas,Campanhas,Carrossel,Interacoes
0,Foto,2021-09-11,2858,16,Loja,N,N,NaN,2874
1,Foto,2021-09-11,2930,28,Loja/Produtos,N,N,NaN,2958
2,Foto,2021-09-11,2807,9,Loja,N,N,NaN,2816
3,Vídeo,2021-09-12,5115,49,Produtos,N,N,NaN,5164
4,Foto,2021-09-13,4392,45,Produtos,S,N,NaN,4437


In [40]:
#Selecionando apenas a coluna Carrossel
base.loc[base.Carrossel.isnull(), 'Carrossel'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: Carrossel, dtype: object

In [41]:
#Atribuindo o valor N para essa coluna
base.loc[base.Carrossel.isnull(), 'Carrossel'] = 'N'

In [42]:
#Verificando novamente os valores de Carrossel 
base.loc[base.Carrossel.isnull()]

,Tipo,Data,Curtidas,Comentários,Tags,Pessoas,Campanhas,Carrossel,Interacoes


Assim, não temos mais valores vazios na coluna Carrossel

In [43]:
#Verificando a descrição estatística da base
base.describe()

,Curtidas,Comentários,Interacoes
count,52.00,52.00,52.00
mean,"12,262.73",189.50,"12,452.23"
std,"8,165.88",170.69,"8,299.39"
min,"2,807.00",9.00,"2,816.00"
25%,"5,492.00",69.50,"5,562.50"
50%,"9,603.00",128.00,"9,773.50"
75%,"17,621.75",265.25,"17,920.75"
max,"37,351.00",852.00,"37,853.00"


### Verificando padrões entre as colunas e informações

Registros com mais e menos curtidas

In [44]:
#Ordenando os valores (mais curtidas)
base.sort_values(by = 'Curtidas', ascending = False).head()

,Tipo,Data,Curtidas,Comentários,Tags,Pessoas,Campanhas,Carrossel,Interacoes
39,Foto,2022-02-17,37351,502,Promoções,S,S,N,37853
30,Reels,2022-01-24,29981,502,Trends,S,S,N,30483
48,Foto,2022-03-20,29084,479,Datas comemorativas/Promoções,S,S,N,29563
33,Foto,2022-02-06,24655,186,Influenciadores,S,S,N,24841
26,Foto,2022-01-08,24585,354,Datas comemorativas,S,S,S,24939


In [45]:
#Ordenando os valores (menos curtidas)
base.sort_values (by = 'Curtidas', ascending = True).head()

,Tipo,Data,Curtidas,Comentários,Tags,Pessoas,Campanhas,Carrossel,Interacoes
2,Foto,2021-09-11,2807,9,Loja,N,N,N,2816
0,Foto,2021-09-11,2858,16,Loja,N,N,N,2874
20,Foto,2021-12-16,2881,29,Produtos,N,N,N,2910
1,Foto,2021-09-11,2930,28,Loja/Produtos,N,N,N,2958
17,Vídeo,2021-11-09,3213,60,Produtos,N,N,N,3273


Em primeiro momentos temos:
- As 5 mais curtidas são postagens com pessoas e de campanhas
- As 5 menos curtidas são postagens sem pessoas e não de campanhas

Vamos nos aprofundar nessas informações

In [46]:
#Para melhor visualização, utilizar um padrão de formatação
pd.options.display.float_format = '{:,.2f}'.format

In [47]:
base.groupby('Pessoas')[['Curtidas', 'Comentários']].mean()

,Curtidas,Comentários
Pessoas,,
N,"4,256.67",52.83
S,"14,664.55",230.50


In [48]:
base.groupby('Campanhas')[['Curtidas', 'Comentários']].mean()

,Curtidas,Comentários
Campanhas,,
N,"7,928.33",123.17
S,"18,173.27",279.95


- Podemos analisar que postagens com pessoas fazem toda diferença, sendo o engajamento tanto de curtidas como de comentários 3 vezes maior de quando não tem pessoas.
- Podemos observar que postagens de campanhas também tem engajamento melhor, 2 vezes maior.

In [49]:
base.groupby(['Pessoas', 'Campanhas'])[['Curtidas', 'Comentários']].mean()

Curtidas  Comentários
Pessoas Campanhas                       
N       N          3,937.50        53.90
        S          5,852.50        47.50
S       N          9,923.75       157.80
        S         19,405.35       303.20

- A média quando a postagem tem pessoas E campanhas é muito alta.

Analisando os tipos de postagens

In [50]:
base.groupby(['Pessoas', 'Campanhas', 'Tipo'])[['Curtidas', 'Comentários']].mean()

Curtidas  Comentários
Pessoas Campanhas Tipo                        
N       N         Foto   2,869.00        20.50
                  Reels  5,934.50        98.00
                  Vídeo  4,007.50        65.25
        S         Foto   5,852.50        47.50
S       N         Foto  10,815.29       159.93
                  IGTV   6,833.40       133.60
                  Reels 12,894.00       249.00
        S         Foto  19,105.38       284.19
                  Reels 24,801.00       388.50
                  Vídeo 16,409.50       370.00

In [51]:
base[base.Tipo == 'Vídeo']

,Tipo,Data,Curtidas,Comentários,Tags,Pessoas,Campanhas,Carrossel,Interacoes
3,Vídeo,2021-09-12,5115,49,Produtos,N,N,N,5164
9,Vídeo,2021-09-28,4056,81,Produtos,N,N,N,4137
15,Vídeo,2021-11-04,3646,71,Produtos,N,N,N,3717
17,Vídeo,2021-11-09,3213,60,Produtos,N,N,N,3273
24,Vídeo,2021-12-30,17600,383,Trends,S,S,N,17983
37,Vídeo,2022-02-13,15219,357,Datas comemorativas,S,S,N,15576


- Os vídeos que possuem boas médias de curtidas e comentários são com pessoas e campanhas, já os que tem média baixa são sem pessoas e camapanhas
- Os vídeos possuem uma média baixa, porém quando esse vídeo é com pessoas e campanhas é alta. Logo, o problema não é o vídeo em si
- O IGTV possui uma média baixa mesmo tendo pessoas

Analisando o impacto da coluna Carrossel (o único tipo de postagem que permite carrossel são fotos)

In [52]:
base[base.Tipo == 'Foto'].groupby('Carrossel')[['Curtidas', 'Comentários']].mean()

,Curtidas,Comentários
Carrossel,,
N,"13,776.36",208.57
S,"11,817.88",140.38


- A média de carrossel, inicialmente, não impacta tanto o resultado. A média é um pouco melhor quando não utilizamos carrossel

##### CONCLUSÕES INICIAS
- Postagens incluindo pessoas engajam muito mais
- Postagens em épocas de campanhas também possuem melhor engajamento
- O carrossel não foi um diferencial para o engajamento da marca

### Analisando Tags

In [53]:
base.groupby('Tags')['Curtidas'].mean()

Tags
Datas comemorativas                 17,975.00
Datas comemorativas/Promoções       29,084.00
Dicas de como usar/Novos Produtos    5,703.50
Dicas de como usar/Produtos          7,586.67
Influenciadores                     15,197.29
Loja                                 2,832.50
Loja/Produtos                        2,930.00
Novos Produtos                      11,619.57
Produtos                             5,666.92
Promoções                           26,645.50
Trends                              22,400.67
Trends/Produtos                     12,894.00
Name: Curtidas, dtype: float64

In [54]:
#Ajustando as linhas que possuem duas tags
base.Tags = base.Tags.str.split('/')
base = base.explode('Tags')
base.head()

,Tipo,Data,Curtidas,Comentários,Tags,Pessoas,Campanhas,Carrossel,Interacoes
0,Foto,2021-09-11,2858,16,Loja,N,N,N,2874
1,Foto,2021-09-11,2930,28,Loja,N,N,N,2958
1,Foto,2021-09-11,2930,28,Produtos,N,N,N,2958
2,Foto,2021-09-11,2807,9,Loja,N,N,N,2816
3,Vídeo,2021-09-12,5115,49,Produtos,N,N,N,5164


In [55]:
base.groupby('Tags')['Curtidas'].mean()

Tags
Datas comemorativas   20,752.25
Dicas de como usar     6,833.40
Influenciadores       15,197.29
Loja                   2,865.00
Novos Produtos        10,304.89
Produtos               6,269.82
Promoções             27,458.33
Trends                20,024.00
Name: Curtidas, dtype: float64

In [56]:
base.groupby('Tags')[['Curtidas', 'Comentários']].mean().sort_values('Curtidas', ascending = False)

,Curtidas,Comentários
Tags,,
Promoções,"27,458.33",531.00
Datas comemorativas,"20,752.25",343.50
Trends,"20,024.00",352.25
Influenciadores,"15,197.29",161.71
Novos Produtos,"10,304.89",198.56
Dicas de como usar,"6,833.40",133.60
Produtos,"6,269.82",94.12
Loja,"2,865.00",17.67


- Postagens com a tag Promoção são as que mais engajam 
- Datas comemorativas e Trends também possuem um bom engajamento

In [60]:
base.groupby(['Pessoas', 'Tags'])[['Curtidas', 'Comentários']].mean().sort_values('Curtidas', ascending = False)

Curtidas  Comentários
Pessoas Tags                                      
S       Promoções           27,458.33       531.00
        Datas comemorativas 20,752.25       343.50
        Trends              20,024.00       352.25
        Sem tag             15,347.88       207.75
        Influenciadores     15,197.29       161.71
        Novos Produtos      10,923.12       215.62
        Produtos             8,316.38       131.62
        Dicas de como usar   6,833.40       133.60
N       Novos Produtos       5,359.00        62.00
        Produtos             4,450.67        60.78
        Loja                 2,865.00        17.67

In [61]:
base.groupby(['Campanhas', 'Tags'])[['Curtidas', 'Comentários']].mean().sort_values('Curtidas', ascending = False)

Curtidas  Comentários
Campanhas Tags                                      
S         Promoções           33,217.50       490.50
          Trends              22,400.67       386.67
          Datas comemorativas 20,752.25       343.50
          Influenciadores     18,715.40       197.60
          Sem tag             16,850.75       257.75
N         Promoções           15,940.00       612.00
          Sem tag             13,845.00       157.75
          Trends              12,894.00       249.00
S         Novos Produtos      11,040.67       323.00
N         Novos Produtos       9,937.00       136.33
S         Produtos             9,074.00        67.50
N         Dicas de como usar   6,833.40       133.60
          Influenciadores      6,402.00        72.00
          Produtos             5,895.93        97.67
          Loja                 2,865.00        17.67

## CONCLUSÕES FINAIS
- Ter o rosto de outras pessoas é fundamental para um bom engajamento na publicação (em todas as tags, quando havia o rosto, o resultado foi muito melhor)
- Criar camapnhas ajuda muito na divulgação da marca
- Promoções tiveram um desempenho absurdamente maior que qualquer outra tag (porém, pode ter custo para a loja e deve ser analisado)
- Usar conteúdos que estão em trend também ajudam na divulgação da marca, mesmo a trend sendo de outros nichos
- A melhor maneira de mostrar produtos é através de outras pessoas utilizando-os, e se possível em campanhas de datas especiais
- Para novos produtos a inclusão de pessoas é mais crítica ainda, sendo quase o dobro quando há um rosto junto ao produto
- Não podemos afirmar que a tag 'Loja' é ruim até testarmos essa tag incluindo pessaos ou em uma camapanha. Vale o teste para verificar